In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from UrbanSoundDataset import UrbanSoundDataset
from FeedForward import CNNNetwork

d:\pribadi\test\audio\UrbanSoundDataset.py:6: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [3]:
class_mapping = [
    "air_conditioner",
    "car_horn",
    "children_playing",
    "dog_bark",
    "drilling",
    "engine_idling",
    "gun_shot",
    "jackhammer",
    "siren",
    "street_music"
]

In [4]:
def predict(model, input, target, class_mapping):
    model.eval()
    with torch.no_grad():
        predictions = model(input)
        # Tensor (1, 10) -> [ [0.1, 0.01, ..., 0.6] ]
        predicted_index = predictions[0].argmax(0)
        predicted = class_mapping[predicted_index]
        expected = class_mapping[target]
    return predicted, expected

In [5]:
cnn = CNNNetwork().to(device)

In [6]:
state_dict = torch.load("models/cnn.pth")

In [7]:
cnn.load_state_dict(state_dict)

<All keys matched successfully>

In [8]:
import os
ANNOTATION_FILE = "./data/UrbanSound8K/metadata/UrbanSound8K.csv"
AUDIO_DIR = os.path.join("data", "UrbanSound8K", "audio")
SAMPLE_RATE = 22050
NUM_SAMPLES = 22050
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001

In [9]:
import torchaudio

mel_spectrogram = torchaudio.transforms.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_fft=1024,
    hop_length=512,
    n_mels=64
)

In [10]:
usd = UrbanSoundDataset(ANNOTATION_FILE, AUDIO_DIR, mel_spectrogram, SAMPLE_RATE, NUM_SAMPLES, device)

In [11]:
predicted_list = []
data_len = 100
data_right = 0
for i in range(100):
    input, target = usd[i][0], usd[i][1]
    input.unsqueeze_(0)
    predicted, expected = predict(cnn, input, target, class_mapping)
    predicted_list.append({
        "expected": expected,
        "predicted": predicted
    })
    if predicted == expected:
        data_right += 1

print("Accuracy: {:.2f}".format(data_right / data_len))

Accuracy: 0.28
